In [15]:
import numpy as np
import scipy.linalg as la
import time
import bezout_2 as bz

#TEX_DIR = '/home/jp/Documents/Bezout/bezout/tex/txt'
TEX_DIR = '../tex/txt'

deg = [1]*6

with open(TEX_DIR+'/deg.txt', 'w') as f:
    f.write(str(deg))
    
t = 5
m = 16000
n = len(deg)

Field = QQ
R = PolynomialRing(Field, 'x', n)
x = R.gens()
xx = [x[0]**0] + list(x)

fshape = [d+1 for d in deg]
dx = [(i+1)*deg[i] for i in range(n)]
dy = [(n-i)*deg[i] for i in range(n)]
fn, Dx, Dy = factorial(n), prod(dx), prod(dy)
with open(TEX_DIR+'/Dx.txt', 'w') as f:
    f.write("{0:d}".format(Dx))


P = [bz.rand_poly(n-1, m, deg, t, x) for i in range(n)] + xx
#P = load('P.sobj')

save(P, 'P')
bz.P2txt(n, deg, P, TEX_DIR)
F = [bz.poly2prism(fshape, p) for p in P]

TEX_DIR

t = time.clock()
Gx, Gy, Hx, Hy = bz._GH(n, fn, deg, dx, dy)
H, K = bz._HK(n, Hx, Hy)
J = bz._J(Dx, Dy, F, n, fshape, dx, dy, Gx, Gy)
C = bz._C(n, Dx, Dy, J)
B = bz._B(n, Dx, Dy, H, K, C)

construction_B_time = time.clock() - t
with open(TEX_DIR+'/construction_B_time.txt', 'w') as f:
    f.write("{0:.4f}".format(construction_B_time))

#B = bz.block_triang(n, Dx, Dy, dx, dy, deg, B)
BB = []
for k in range(n+1):
        Bk = matrix(Field, B[k])
        BB.append(Bk[:, :])
bz.bz2txt(n, TEX_DIR, BB)

"""
computing rank of B0
"""
t = time.clock()
r0 = rank(BB[0])
rank_B0_time = time.clock() - t

with open(TEX_DIR+'/rank_B0.txt', 'w') as f:
    f.write("{0:d}".format(r0))
with open(TEX_DIR+'/rank_B0_time.txt', 'w') as f:
    f.write("{0:.4f}".format(rank_B0_time))
print("sage_rank = {0:d}".format(r0))

bb = []
for k in range(n+1):
    bb.append(np.array(BB[k], dtype=float))

b0 = bb[0]
numpy_rank = np.linalg.matrix_rank(b0)
print("numpy_rank = {0:d}".format(numpy_rank))

sage_rank = 682
numpy_rank = 682


In [2]:
t = time.clock()
GB, grobner_dim = bz.compute_grobner(R, P, n)
print grobner_dim
grobner_time = time.clock() - t
bz.gb2txt(TEX_DIR, GB)

with open(TEX_DIR+'/grobner_dim.txt', 'w') as f:
    f.write("{0}".format(grobner_dim))

with open(TEX_DIR+'/grobner_time.txt', 'w') as f:
    f.write("{0:.4f}".format(grobner_time))

53


In [17]:
B0Y = BB[0]
Y = bz.find_Y(BB, B0Y, Field, n)

BBt = []
for k in range(n+1):
    BBt.append(BB[k].transpose())
B0Yt = B0Y.transpose()
X = bz.find_Y(BBt, B0Yt, Field, n)

print "-"*10
Y_ortho = Y.right_kernel_matrix()
bb0y = BB[0]*Y_ortho.transpose()

X_ortho = X.right_kernel_matrix()
xbb0 = X_ortho*BB[0]

xbb0y = X_ortho*BB[0]*Y_ortho.transpose()
bezout_exact_dim = rank(xbb0y)
print("bezout_exact_dim = {0:d}".format(bezout_exact_dim))

720
728
730
720
728
730
----------
680


In [18]:
XBBY = []
for k in range(n+1):
    XBBY.append(X_ortho*BB[k]*Y_ortho.transpose())

bezout_exact_dim = 680


In [19]:
K_ortho = XBBY[0].kernel().basis_matrix().right_kernel_matrix()
N_ortho = XBBY[0].right_kernel_matrix().right_kernel_matrix()
#N_ortho = N.kernel().basis_matrix().transpose()

KBBN = []
for k in range(n+1):
    KBBN.append(K_ortho*XBBY[k]*N_ortho.transpose())

In [20]:
Field = GF(next_prime(200))

BBf = []
for k in range(n+1):
        Bk = matrix(Field, KBBN[k])
        BBf.append(Bk[:, :])
        
if rank(BBf[0]) == bezout_exact_dim:
    XX = []
    for k in range(n):
        xx = BBf[0].solve_right(BBf[k+1])
        XX.append(xx)
    Pf = [bz.P2field(p, Field) for p in P]
    test_XX = [bz.X2p(XX, Field, p) for p in Pf[:n]]
    #f.write("test_XX = {0:s}".format(test_XX))
    print("test_XX = {0:s}".format(test_XX))
else:
        print "rank deficiency !"

test_XX = [(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [21]:
bb = []
for k in range(n+1):
    bb.append(np.array(KBBN[k], dtype=float))
roots = bz.BB2roots(bb)
test_roots = bz.roots_test(P, x, roots)
test_roots

array([  3.23425902e-09,   2.26751291e-09,   4.92232770e-08,
         3.69059772e-10,   9.05263463e-09,   2.92390547e-09,
         1.28737135e-10,   9.73419415e-12,   1.75941528e-12,
         1.46970922e-11,   7.17423507e-10,   6.45098762e-10,
         2.33080753e-09,   9.11008780e-11,   8.68381992e-11,
         8.31743128e-10,   1.03187186e-09,   2.28164347e-10,
         1.48532740e-11,   1.47715427e-11,   5.29413377e-09,
         1.36567658e-08,   1.73943151e-10,   2.31222719e-10,
         1.24896696e-08,   2.30834542e-10,   3.00598425e-10,
         5.54742105e-10,   6.88292806e-10,   2.97786362e-10,
         5.92296708e-12,   1.06334808e-10,   1.08139902e-10,
         3.84462908e-11,   2.24282425e-10,   3.39834346e-11,
         2.98496208e-08,   6.89593174e-10,   5.37876459e-10,
         2.88559024e-10,   7.84314870e-09,   3.59011244e-10,
         1.03947685e-11,   5.30919950e-09,   9.67916914e-12,
         4.44346364e-11,   6.84458276e-11,   4.43763957e-11,
         1.31513994e-10,

In [22]:
hist, bin_edges = np.histogram(np.log10(test_roots), bins='scott')
hist, bin_edges

(array([ 29, 101, 141, 112,  92,  61,  43,  33,  30,  17,   7,   5,   3,
          3,   2,   0,   1]),
 array([-11.75463164, -11.44277515, -11.13091866, -10.81906216,
        -10.50720567, -10.19534918,  -9.88349269,  -9.5716362 ,
         -9.2597797 ,  -8.94792321,  -8.63606672,  -8.32421023,
         -8.01235373,  -7.70049724,  -7.38864075,  -7.07678426,
         -6.76492777,  -6.45307127]))